In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 17.7 MB/s eta 0:00:0000:01


In [4]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from ultralytics.utils.downloads import download


def convert_box(size, box):
    """
    Convert VisDrone box to YOLO xywh format.
    :param size: (width, height) of the image
    :param box: (x_min, y_min, width, height) in VisDrone format
    :return: Normalized YOLO format (x_center, y_center, width, height)
    """
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh


def convert_annotations_to_yolo(dir):
    """
    Convert VisDrone annotations to YOLO format.
    :param dir: Directory containing images and annotations
    """
    labels_dir = dir / 'labels'
    labels_dir.mkdir(parents=True, exist_ok=True)  # Create labels directory if it doesn't exist

    annotation_files = list((dir / 'annotations').glob('*.txt'))
    pbar = tqdm(annotation_files, desc=f'Converting annotations in {dir.name}')
    for annotation_file in pbar:
        try:
            # Get corresponding image size
            image_file = (dir / 'images' / annotation_file.stem).with_suffix('.jpg')
            img_size = Image.open(image_file).size

            # Process annotations
            lines = []
            with open(annotation_file, 'r') as file:
                for row in [x.split(',') for x in file.read().strip().splitlines()]:
                    if row[4] == '0':  # Skip 'ignored regions' (class 0 in VisDrone)
                        continue
                    cls = int(row[5]) - 1  # Adjust class index to be 0-based
                    box = convert_box(img_size, tuple(map(int, row[:4])))
                    lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")

            # Write to YOLO label file
            label_file = labels_dir / f"{annotation_file.stem}.txt"
            with open(label_file, 'w') as label_out:
                label_out.writelines(lines)
        except Exception as e:
            pbar.write(f"Error processing {annotation_file}: {e}")


def download_and_prepare_visdrone(dataset_path):
    """
    Download VisDrone dataset and convert annotations to YOLO format.
    :param dataset_path: Root directory for the dataset
    """
    urls = [
        'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-train.zip',
        'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-val.zip',
        'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-dev.zip',
        'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-challenge.zip',
    ]

    dataset_dir = Path(dataset_path)
    download(urls, dir=dataset_dir, curl=True, threads=4)  # Download datasets

    # Convert annotations to YOLO format for each split
    for split in ['VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev']:
        convert_annotations_to_yolo(dataset_dir / split)


if __name__ == "__main__":
    dataset_root = '/kaggle/working/VisDrone'
    download_and_prepare_visdrone(dataset_root)
    print("Dataset download and conversion completed.")


curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
Converting annotations in VisDrone2019-DET-val: 100%|██████████| 548/548 [00:00<00:00, 1611.78it/s]
Converting annotations in VisDrone2019-DET-test-dev: 100%|██████████| 1610/1610 [00:00<00:00, 1918.00it/s]

Dataset download and conversion completed.


Visdrone to coco

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [13]:
!pip install imagesize


In [14]:
import glob
import imagesize
import os
import json
from tqdm import tqdm

def convert(dir_data, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    train_data = dir_data + '/VisDrone2019-DET-train/'
    val_data = dir_data + '/VisDrone2019-DET-val/'
    test_data = dir_data + '/VisDrone2019-DET-test-dev/'
    loops = [train_data, val_data, test_data]
    
    for l in loops:
        print('Processing ', l)
        dict_coco = {}
        dir_imgs = './images/'

        # Key: images
        print('Processing images')
        dict_image_and_id = {}
        dict_coco['images'] = []
        img_id = 0
        for img in tqdm(glob.glob(l + dir_imgs + '*')):
            width, height = imagesize.get(img)
            file_name_save = os.path.split(img)[-1]
            dict_coco['images'].append({
                "id": img_id,
                "license": 1,
                "height": height,
                "width": width,
                "file_name": file_name_save
            })
            dict_image_and_id[file_name_save] = img_id
            img_id += 1

        # Key: annotations
        print('Processing annotations')
        dir_labels = '/annotations/'
        dict_coco['annotations'] = []
        anno_id = 0
        for file_txt in tqdm(glob.glob(l + dir_labels + '*.txt')):
            annotations = open(file_txt, 'r').read().strip().split('\n')
            for i in range(len(annotations)):
                annotations[i] = annotations[i].split(',')
            for detection in annotations:
                category_id = int(detection[5])
                bbox = [int(detection[0]), int(detection[1]), int(detection[2]), int(detection[3])]
                area = int(detection[2]) * int(detection[3])
                img_name = os.path.splitext(os.path.split(file_txt)[-1])[0] + '.jpg'
                image_id = dict_image_and_id[img_name]

                dict_coco['annotations'].append({
                    "id": anno_id,
                    "image_id": image_id,
                    "category_id": category_id,
                    "bbox": bbox,
                    "area": area,
                    "iscrowd": 0,
                    "ignore": 0
                })
                anno_id += 1

        # Key: categories
        dict_coco['categories'] = [
            {"id": 1, "name": "pedestrian", "supercategory": "none"},
            {"id": 2, "name": "people", "supercategory": "none"},
            {"id": 3, "name": "bicycle", "supercategory": "none"},
            {"id": 4, "name": "car", "supercategory": "none"},
            {"id": 5, "name": "van", "supercategory": "none"},
            {"id": 6, "name": "truck", "supercategory": "none"},
            {"id": 7, "name": "tricycle", "supercategory": "none"},
            {"id": 8, "name": "awning-tricycle", "supercategory": "none"},
            {"id": 9, "name": "bus", "supercategory": "none"},
            {"id": 10, "name": "motor", "supercategory": "none"},
            {"id": 11, "name": "others", "supercategory": "none"}
        ]

        # Save to JSON in the specified output directory
        dataset_type = l.split('-')[-1][:-1]  # e.g., train, val, or test
        output_file = os.path.join(output_dir, f'annotations_VisDrone_{dataset_type}.json')
        with open(output_file, 'w') as f:
            json.dump(dict_coco, f)
        print(f"Saved {output_file}")

# Set the data directory and output directory
data_dir = '/kaggle/working/VisDrone'  # Replace with the actual path to your dataset
output_dir = './output_annotations'  # Replace with the desired output directory

# Run the conversion
convert(data_dir, output_dir)


Processing  /kaggle/working/VisDrone/VisDrone2019-DET-train/
Processing images


100%|██████████| 6471/6471 [00:00<00:00, 42648.58it/s]


Processing annotations


100%|██████████| 6471/6471 [00:02<00:00, 2558.73it/s]


Saved ./output_annotations/annotations_VisDrone_train.json
Processing  /kaggle/working/VisDrone/VisDrone2019-DET-val/
Processing images


...

Saved ./output_annotations/annotations_VisDrone_val.json
Processing  /kaggle/working/VisDrone/VisDrone2019-DET-test-dev/
Processing images


100%|██████████| 1610/1610 [00:00<00:00, 46062.64it/s]


Processing annotations


100%|██████████| 1610/1610 [00:00<00:00, 3125.71it/s]


Saved ./output_annotations/annotations_VisDrone_dev.json


In [15]:

from IPython.display import clear_output

# Clone my repo
!git clone https://github.com/NyanSwanAung/Object-Detection-Using-DETR-CustomDataset.git
%cd Object-Detection-Using-DETR-CustomDataset/
clear_output()

In [16]:
DATASET_PATH = '/kaggle/working/VisDrone'

TRAIN_IMG = '/kaggle/working/VisDrone/VisDrone2019-DET-train/images'
TRAIN_JSON = '/kaggle/working/output_annotations/annotations_VisDrone_train.json'

VAL_IMG = '/kaggle/working/VisDrone/VisDrone2019-DET-val/images'
VAL_JSON = '/kaggle/working/output_annotations/annotations_VisDrone_val.json'

In [17]:
import os 
print(os.path.exists(DATASET_PATH))
print(os.path.exists(TRAIN_IMG))
print(os.path.exists(TRAIN_JSON))
print(os.path.exists(VAL_IMG))
print(os.path.exists(VAL_JSON))

True
True
True
True
True


In [21]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'



  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-uxyw6naj
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-uxyw6naj
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=101098 sha256=935f5312ffedd6d3ddf20c6ff3bd0364bde21e9ffb6925b98fa31ba2aedc20d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-jz8ghjnq/wheels/39/61/b4/480fbddb4d3d6bc34083e7397bc6f5d1381f79acc68e9f3511
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.8
    Uninstalling pycocotools-2.0.8:
      Successfully uninstalled pycocotools-2.0.8


In [23]:
import os
import shutil

base_dir = "/kaggle/working/visdrone"
annotations_dir = os.path.join(base_dir, "annotations")
train_images_dir = os.path.join(base_dir, "train2017")
val_images_dir = os.path.join(base_dir, "val2017")

os.makedirs(annotations_dir, exist_ok=True)
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)

train_images_src = "/kaggle/working/VisDrone/VisDrone2019-DET-train/images"
val_images_src = "/kaggle/working/VisDrone/VisDrone2019-DET-val/images"
train_annotations_src = "/kaggle/working/output_annotations/annotations_VisDrone_train.json"
val_annotations_src = "/kaggle/working/output_annotations/annotations_VisDrone_val.json"

for filename in os.listdir(train_images_src):
    src_path = os.path.join(train_images_src, filename)
    dest_path = os.path.join(train_images_dir, filename)
    shutil.copy(src_path, dest_path)

for filename in os.listdir(val_images_src):
    src_path = os.path.join(val_images_src, filename)
    dest_path = os.path.join(val_images_dir, filename)
    shutil.copy(src_path, dest_path)

# Copy annotation files
shutil.copy(train_annotations_src, os.path.join(annotations_dir, "instances_train2017.json"))
shutil.copy(val_annotations_src, os.path.join(annotations_dir, "instances_val2017.json"))

print("Folder structure created and files copied successfully.")


Folder structure created and files copied successfully.


In [28]:

!python -m torch.distributed.launch --nproc_per_node=2 main.py --coco_path /kaggle/working/visdrone 
 

/opt/conda/lib/python3.10/site-packages/torch/distributed/launch.py:208: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  main()
/opt/conda/bin/python: can't open file '/kaggle/working/Object-Detection-Using-DETR-CustomDataset/main.py': [Errno 2] No such file or directory
/opt/conda/bin/python: can't open file '/kaggle/working/Object-Detection-Using-DETR-CustomDataset/main.py': [Errno 2] No such file or directory
E1211 06:06:26.216000 133572998035264 torch/distributed/elastic/multiprocessing/api.py:833] failed (exitcode: 2) local_rank: 0 (pid: 631) of binary: /opt/conda/bin/python
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py"